In [8]:
import tkinter as tk
from tkinter import filedialog, messagebox, Label, Button, OptionMenu, StringVar
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model
import joblib

# Dictionary to map model names to file paths
models = {
    "CNN": "C:/Users/DELL/Desktop/Res_project/PCOS/PCOS/models/cnn_model2.h5",
    "SVM": "C:/Users/DELL/Desktop/Res_project/PCOS/PCOS/models/SVM.joblib",
}

def load_image(img_path, model_type):
    """ Load and prepare image for prediction """
    img = Image.open(img_path).resize((224, 224))
    img_array = np.array(img) / 255.0

    if model_type == "CNN":
        # For CNN model, reshape image array to match the input shape (samples, height, width, channels)
        img_array = img_array.reshape(1, 224, 224, 3)
    elif model_type == "SVM":
        # For SVM model, flatten image array to be 1-dimensional
        img_array = img_array.flatten().reshape(1, -1)

    return img_array


def select_image():
    """ Handle image selection and update display """
    global img_path, image_panel
    img_path = filedialog.askopenfilename(title='Open Image', filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if img_path:
        img = Image.open(img_path).resize((250, 250))  # Resize for display
        img = ImageTk.PhotoImage(img)
        image_panel.configure(image=img)
        image_panel.image = img
        predict_button.config(state='normal')

def predict():
    """ Make prediction using selected model and display result """
    if not img_path or not model_var.get():
        messagebox.showerror("Error", "Please select an image and a model.")
        return
    
    model_type = model_var.get()
    model_path = models[model_type]
    
    if model_type == "SVM":
        model = joblib.load(model_path)  # Load SVM model
    else:
        model = load_model(model_path)  # Load CNN model
    
    img = load_image(img_path, model_type)  # Pass model_type to load_image
    
    if model_type == "SVM":
        prediction = model.predict(img)
        pred_class = prediction[0]
    else:
        prediction = model.predict(img)
        pred_class = np.argmax(prediction, axis=-1)[0]

    result = "Infected" if pred_class == 0 else "Not Infected"
    result_label.config(text=f"Prediction ({model_type}): {result}")

  
# Create main window
root = tk.Tk()
root.title("PCOS Ultrasound Prediction")
root.geometry("550x600")

# Frame for image display
image_panel = Label(root)
image_panel.pack(padx=10, pady=10)

# Button to open image
open_button = Button(root, text='Open Image', command=select_image)
open_button.pack()

# Dropdown menu for model selection
model_var = StringVar()
model_var.set("Select Model")
model_menu = OptionMenu(root, model_var, *models.keys())
model_menu.pack()

# Button to start prediction
predict_button = Button(root, text="Predict", state='disabled', command=predict)
predict_button.pack(pady=20)

# Label to display prediction result
result_label = Label(root, text="", font=('Helvetica', 14))
result_label.pack(pady=20)

root.mainloop()


1/1 [==============================] - 0s 164ms/step
